In [1]:
import dev_ga_base

Poblacion creada con 20000 individuos
Inicio del proceso iterativo

###
Paso  0
Puntaje: 4
Fallo: [5, 7, 8, 11, 14]
Mejor:  ['blue', 'Developer', 'JavaScript', 'Cassandra', 'Vim']
Mejor:  ['yellow', 'Analyst', 'C#', 'Neo4j', 'Sublime Text']
Mejor:  ['white', 'Hacker', 'Python', 'MongoDB', 'Notepad++']
Mejor:  ['green', 'Engineer', 'Java', 'HBase', 'Brackets']
Mejor:  ['red', 'Mathematician', 'C++', 'Redis', 'Atom']

###
Paso  1
Puntaje: 6
Fallo: [3, 4, 9, 11]
Mejor:  ['blue', 'Developer', 'C#', 'HBase', 'Brackets']
Mejor:  ['yellow', 'Analyst', 'Python', 'Cassandra', 'Atom']
Mejor:  ['red', 'Mathematician', 'JavaScript', 'Neo4j', 'Sublime Text']
Mejor:  ['white', 'Hacker', 'Java', 'Redis', 'Vim']
Mejor:  ['green', 'Engineer', 'C++', 'MongoDB', 'Notepad++']

###
Paso  2
Puntaje: 8
Fallo: [3, 4, 5]
Mejor:  ['blue', 'Developer', 'Java', 'Redis', 'Brackets']
Mejor:  ['yellow', 'Analyst', 'JavaScript', 'Cassandra', 'Vim']
Mejor:  ['white', 'Hacker', 'C#', 'HBase', 'Notepad++']
Mejor:  ['gre

# Resultado

Tiempo transcurrido **00:00:09.69**

- Fitness: 14 
 
- Individuo ['010', '101', '011', '101', '011', '001', '001', '100', '100', '010', '101', '010', '001', '001', '101', '100', '100', '010', '011', '100', '011', '011', '101', '010', '001'] 

- Individuo [['blue', 'Developer', 'Java', 'Redis', 'Vim'], ['red', 'Mathematician', 'C++', 'Neo4j', 'Sublime Text'], ['yellow', 'Hacker', 'Python', 'Cassandra', 'Notepad++'], ['white', 'Analyst', 'C#', 'HBase', 'Atom'], ['green', 'Engineer', 'JavaScript', 'MongoDB', 'Brackets']]


```python
[
    ['blue', 'Developer', 'Java', 'Redis', 'Vim'],
    ['red', 'Mathematician', 'C++', 'Neo4j', 'Sublime Text'],
    ['yellow', 'Hacker', 'Python', 'Cassandra', 'Notepad++'],
    ['white', 'Analyst', 'C#', 'HBase', 'Atom'],
    ['green', 'Engineer', 'JavaScript', 'MongoDB', 'Brackets']
]
```

# Analisis

## Que encontramos?

- Generar una población inicial conforme a las reglas del problema mejora el funcionamiento del algoritmo contra generar una población con caracteristicas random.

- En una instancia utilizamos Multithreading para agilizar el procesamiento de datos. Luego, al aplicar algunas mejoras en el algoritmo no fue necesario el uso.

- Hacer un crossover 'aleatorio' para generar un hijo nuevo, no es lo mejor ya que puede generar individuos invalidos. Fuimos por la opcion de controlar el crossover para que intercambie caracteristicas de los 2 padres y genere un hijo valido.

- Hacer una mutación 'aleatoria' no es de lo mejor. Encontramos un buen resultado haciendo un swap de caracteristicas para no generar un individuo invalido luego de la mutación.

## Población inicial

La idea para la población inicial es generar individuos validos con respecto a la logica del problema. Un ejemplo de esto seria no crear 2 casas con el mismo color. El codigo utilizado para realizar este proceso es:

```python
def createRandomChromosome(self):
    colors, prefession, languaje, database, editor = ([0,1,2,3,4] for _ in range(5))

    chromosome = []

    for _ in range(0, 5):
        rn = random.randint(0, len(colors)-1)
        chromosome.append(colors[rn])
        del colors[rn]

        rn = random.randint(0, len(prefession)-1)
        chromosome.append(prefession[rn])
        del prefession[rn]

        rn = random.randint(0, len(languaje)-1)
        chromosome.append(languaje[rn])
        del languaje[rn]

        rn = random.randint(0, len(database)-1)
        chromosome.append(database[rn])
        del database[rn]

        rn = random.randint(0, len(editor)-1)
        chromosome.append(editor[rn])
        del editor[rn]

    return chromosome
```

## Mutación

Para la mutación de invididuos utilizamos un algoritmo que realice un swap entre mismas caracteristicas dentro del individuo. Ejemplo, hacer un swap del color de la casa1 con el color de la casa4.

```python
def mutate(self):
    ''' muta un fenotipo haciendo swap'''
    ''' se puede hacer esto porque cambié la función de inicialización de la población para que no contenga repetidos'''

    col = random.randint(0, 4) #Tipo de característica
    row1 = random.randint(0, 4) #Valor de la caracteristica 1
    row2 = random.randint(0, 4) #Valor de la caracteristica 2

    while row1 == row2:
        row2 = random.randint(0, 4)

    auxSwapValue = self.chromosome[row1*5+col]
    self.chromosome[row1*5+col] = self.chromosome[row2*5+col]
    self.chromosome[row2*5+col] = auxSwapValue
```

## Fitness

La función de fitness lo que hace es validar las reglas de logica del problema. En total son 14 reglas que debe cumplir cada individuo. En caso de que un individuo no cumpla con una de las reglas, no sumara puntaje.

```python
def fitness_function(self):
    ''' calcula el valor de fitness del cromosoma segun el problema en particular '''

    self.score = 0

    ok_score = 1
    fail_score = -1
    punish_score = -1

    matrix = [[0 for x in range(5)] for x in range(5)]

    for i in range(0, 5):
        for j in range(0, 5):
            matrix[i][j] = self.chromosome[j*5+i]

    # 2. El Matematico vive en la casa roja.
    try:
        i = matrix[PROFESSION_INDEX].index(0)
        if matrix[COLOR_INDEX][i] == 0:
            self.score += ok_score
        else:
            self.score += fail_score
    except:
        self.score += punish_score
    ...
    ...
    ...
```

## Crossover

La idea del crossover es, dado 2 individuos padres, generar un hijo el cual no rompa las reglas de logica del problema. Es decir, no genere individuos invalidos.

```python
def crossOverPopulation(self, parents):
    next_population = []

    parent_count = len(parents)
    generation_target_count = POPULATION_LEN - PARENT_TO_NEXT_GEN

    while len(next_population) < generation_target_count:
        if (len(next_population) + parent_count) > generation_target_count:
            for progenitor in parents:
                otherProgenitor = parents[random.randint(0, parent_count - 1)]
                next_population.append(self.crossOver(progenitor, otherProgenitor))
                if len(next_population) >= generation_target_count:
                    break
        else:
            for progenitor in parents:
                otherProgenitor = parents[random.randint(0, parent_count - 1)]
                next_population.append(self.crossOver(progenitor, otherProgenitor))

    return next_population
```